Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


tách frame

In [ ]:
import cv2
import os

# Đường dẫn tới video
video_path = '/content/drive/MyDrive/pbl4/video/id9_cl.MOV'

# Tạo thư mục lưu các frame
output_dir = '/content/drive/MyDrive/pbl4/frames/lan2/082/cl'
os.makedirs(output_dir, exist_ok=True)

# Mở video
cap = cv2.VideoCapture(video_path)

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Lưu frame thành file ảnh
    frame_filename = os.path.join(output_dir, f"frame_{frame_count:05d}.jpg")
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

cap.release()
print(f"Đã tách {frame_count} frame và lưu vào thư mục '{output_dir}'")


Đã tách 521 frame và lưu vào thư mục '/content/drive/MyDrive/pbl4/frames/lan2/082/cl'


Segmenatation

In [ ]:
import cv2
import torch
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import os
import glob
from torchvision.models.segmentation import deeplabv3_resnet50

# ------------------------- ĐƯỜNG DẪN CƠ BẢN -------------------------

input_root = "/content/drive/MyDrive/pbl4/frames/lan2/082"
output_root = "/content/drive/MyDrive/pbl4/new_setup/082"
folders = ["nm-01", "nm-02", "bg", "cl"]

# ------------------------- MÔ HÌNH SEGMENTATION -------------------------

model = deeplabv3_resnet50(weights="DEFAULT")
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
people_class = 15

# Gaussian blur kernel
blur = torch.FloatTensor([[[[1.0, 2.0, 1.0],
                            [2.0, 4.0, 2.0],
                            [1.0, 2.0, 1.0]]]]) / 16.0
blur = blur.to(device)

# Preprocessing
preprocess = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

# ------------------------- HÀM TẠO MASK -------------------------

def makeSegMask(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    frame_data = torch.from_numpy(img_rgb).float() / 255.0
    input_tensor = preprocess(frame_data.permute(2, 0, 1))
    input_batch = input_tensor.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_batch)['out'][0]

    segmentation = output.argmax(0)
    bgOut = output[0:1]
    a = (1.0 - F.relu(torch.tanh(bgOut * 0.30 - 1.0))).pow(0.5) * 2.0
    people = segmentation.eq(torch.ones_like(segmentation).long().fill_(people_class)).float()
    people = people.unsqueeze(0).unsqueeze(0)

    for _ in range(3):
        people = F.conv2d(people, blur, stride=1, padding=1)

    combined_mask = F.relu(F.hardtanh(a * (people.squeeze().pow(1.5)))).cpu()
    combined_mask = combined_mask.expand(1, 3, -1, -1)
    res = (combined_mask * 255.0).squeeze().byte().permute(1, 2, 0).numpy()
    return res

# ------------------------- XỬ LÝ ẢNH TRONG CÁC FOLDER -------------------------

for folder in folders:
    input_folder = os.path.join(input_root, folder)
    output_folder = os.path.join(output_root, folder)
    os.makedirs(output_folder, exist_ok=True)

    image_files = sorted(glob.glob(os.path.join(input_folder, "*.jpg")))
    if not image_files:
        print(f"❌ Không tìm thấy ảnh trong {input_folder}")
        continue

    for img_path in image_files:
        print(f"➡️  Đang xử lý: {img_path}")
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️  Lỗi đọc ảnh: {img_path}")
            continue

        # Tạo mask
        mask = makeSegMask(img)

        # Chuyển mask thành silhouette
        gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        _, binary_mask = cv2.threshold(gray_mask, 50, 255, cv2.THRESH_BINARY)

        silhouette = np.zeros_like(img)
        silhouette[binary_mask == 255] = (255, 255, 255)

        # Lưu ảnh silhouette
        base_filename = os.path.basename(img_path)
        output_path = os.path.join(output_folder, f"silhouette_{base_filename}")
        cv2.imwrite(output_path, silhouette)

        print(f"✅  Đã lưu: {output_path}")

print("🎉 Hoàn tất xử lý tất cả ảnh trong 3 thư mục.")


➡️  Đang xử lý: /content/drive/MyDrive/pbl4/frames/lan2/082/nm-01/frame_00072.jpg
✅  Đã lưu: /content/drive/MyDrive/pbl4/new_setup/082/nm-01/silhouette_frame_00072.jpg
➡️  Đang xử lý: /content/drive/MyDrive/pbl4/frames/lan2/082/nm-01/frame_00073.jpg
✅  Đã lưu: /content/drive/MyDrive/pbl4/new_setup/082/nm-01/silhouette_frame_00073.jpg
➡️  Đang xử lý: /content/drive/MyDrive/pbl4/frames/lan2/082/nm-01/frame_00074.jpg
✅  Đã lưu: /content/drive/MyDrive/pbl4/new_setup/082/nm-01/silhouette_frame_00074.jpg
➡️  Đang xử lý: /content/drive/MyDrive/pbl4/frames/lan2/082/nm-01/frame_00075.jpg
✅  Đã lưu: /content/drive/MyDrive/pbl4/new_setup/082/nm-01/silhouette_frame_00075.jpg
➡️  Đang xử lý: /content/drive/MyDrive/pbl4/frames/lan2/082/nm-01/frame_00076.jpg
✅  Đã lưu: /content/drive/MyDrive/pbl4/new_setup/082/nm-01/silhouette_frame_00076.jpg
➡️  Đang xử lý: /content/drive/MyDrive/pbl4/frames/lan2/082/nm-01/frame_00077.jpg
✅  Đã lưu: /content/drive/MyDrive/pbl4/new_setup/082/nm-01/silhouette_frame_00

resize 64x64

In [ ]:
from PIL import Image
import os

def resize_images_in_place(folder_path, size=(64, 64)):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
            file_path = os.path.join(folder_path, filename)
            try:
                with Image.open(file_path) as img:
                    resized_img = img.resize(size, Image.Resampling.LANCZOS)
                    resized_img.save(file_path)  # ⚠️ Ghi đè lên ảnh gốc
                    print(f"✅ Đã resize: {file_path}")
            except Exception as e:
                print(f"❌ Lỗi với {file_path}: {e}")

# Duyệt từ 075 đến 077 như yêu cầu
for i in range(78, 83):
    folder = f'{i:03}'  # '075', '076', '077'
    path = f'/content/drive/MyDrive/pbl4/new_set_pkl/{folder}/cl-02/090'
    resize_images_in_place(path)


✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00544.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00553.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00546.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00573.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00558.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00571.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00568.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00565.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00554.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-02/090/silhouette_frame_00562.jpg
✅ Đã resize: /content/drive/MyDrive/pbl4/new_set_pkl/078/cl-

Silhouettes

In [ ]:
import cv2
import torch
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import os
import glob
import time
from torchvision.models.segmentation import deeplabv3_resnet50

# Đường dẫn gốc
base_input_folder = "/content/drive/MyDrive/pbl4/setup/075"
base_output_folder = "/content/drive/MyDrive/pbl4/setup2/075"

# Load model
model = deeplabv3_resnet50(weights="DEFAULT")
model.eval()
people_class = 15

# Gaussian blur
blur = torch.FloatTensor([[[[1.0, 2.0, 1.0],
                            [2.0, 4.0, 2.0],
                            [1.0, 2.0, 1.0]]]]) / 16.0
if torch.cuda.is_available():
    model.to('cuda')
    blur = blur.to('cuda')

# Preprocessing
preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Hàm tạo mask phân đoạn người
def makeSegMask(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    frame_data = torch.from_numpy(img_rgb).float() / 255.0
    input_tensor = preprocess(frame_data.permute(2, 0, 1))
    input_batch = input_tensor.unsqueeze(0).to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        output = model(input_batch)['out'][0]

    segmentation = output.argmax(0)
    bgOut = output[0:1]
    a = (1.0 - F.relu(torch.tanh(bgOut * 0.30 - 1.0))).pow(0.5) * 2.0
    people = segmentation.eq(torch.ones_like(segmentation).long().fill_(people_class)).float()
    people.unsqueeze_(0).unsqueeze_(0)

    for _ in range(3):
        people = F.conv2d(people, blur, stride=1, padding=1)

    combined_mask = F.relu(F.hardtanh(a * (people.squeeze().pow(1.5)))).cpu()
    combined_mask = combined_mask.expand(1, 3, -1, -1)
    res = (combined_mask * 255.0).squeeze().byte().permute(1, 2, 0).numpy()
    return res

# Lặp từ subject 075 -> 124
for subject_id in range(75, 125):
    sub_folder = f"{subject_id:03d}/nm-01/090"
    input_folder = os.path.join(base_input_folder, sub_folder)
    output_folder = os.path.join(base_output_folder, sub_folder)
    os.makedirs(output_folder, exist_ok=True)

    image_files = glob.glob(os.path.join(input_folder, '*.jpg'))

    for input_image_path in image_files:
        print(f"Processing {input_image_path}...")
        img = cv2.imread(input_image_path)
        mask = makeSegMask(img)

        # Threshold mask
        _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        final = cv2.bitwise_and(thresh, img)

        base_filename = os.path.basename(input_image_path)
        mask_output_path = os.path.join(output_folder, f"mask_{base_filename}")
        # silhouette_output_path = os.path.join(output_folder, f"silhouette_{base_filename}")

        cv2.imwrite(mask_output_path, mask)
        # cv2.imwrite(silhouette_output_path, final)

        print(f"Saved to {mask_output_path}")

print("Hoàn thành xử lý tất cả các subject.")


Processing /content/drive/MyDrive/pbl4/frames/lan2/075/nm-01/090/frame_0000.jpg...
Saved to /content/drive/MyDrive/pbl4/pre_pkl/075/nm-01/090/mask_frame_0000.jpg
Processing /content/drive/MyDrive/pbl4/frames/lan2/075/nm-01/090/frame_0001.jpg...
Saved to /content/drive/MyDrive/pbl4/pre_pkl/075/nm-01/090/mask_frame_0001.jpg
Processing /content/drive/MyDrive/pbl4/frames/lan2/075/nm-01/090/frame_0002.jpg...
Saved to /content/drive/MyDrive/pbl4/pre_pkl/075/nm-01/090/mask_frame_0002.jpg
Processing /content/drive/MyDrive/pbl4/frames/lan2/075/nm-01/090/frame_0003.jpg...
Saved to /content/drive/MyDrive/pbl4/pre_pkl/075/nm-01/090/mask_frame_0003.jpg
Processing /content/drive/MyDrive/pbl4/frames/lan2/075/nm-01/090/frame_0004.jpg...
Saved to /content/drive/MyDrive/pbl4/pre_pkl/075/nm-01/090/mask_frame_0004.jpg
Processing /content/drive/MyDrive/pbl4/frames/lan2/075/nm-01/090/frame_0005.jpg...
Saved to /content/drive/MyDrive/pbl4/pre_pkl/075/nm-01/090/mask_frame_0005.jpg
Processing /content/drive/My

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 119.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s e

In [ ]:
!pip install numpy matplotlib PyYAML tabulate termcolor tqdm kornia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 32.1 MB/s eta 0:00:00


In [ ]:
!pip install numpy==1.25.0 matplotlib==3.5.2 PyYAML==6.0.1 tabulate==0.8.10 termcolor==2.4.0 tqdm==4.64.0 kornia

In [ ]:
%cd /content/drive/MyDrive/pbl4/OpenGait

/content/drive/MyDrive/pbl4/OpenGait


In [ ]:
types = ['nm-01', 'nm-02', 'bg-01', 'bg-02', 'cl-01', 'cl-02']

for i in range(78, 83):
    folder = f'{i:03}'  # Định dạng số thành chuỗi 3 chữ số, ví dụ 75 -> '075'
    for t in types:
        input_path = f'/content/drive/MyDrive/pbl4/new_set_pkl/{folder}/{t}/090'
        output_path = f'/content/drive/MyDrive/pbl4/new_pkl/{folder}/{t}/090'
        !python datasets/pretreatment.py --input_path "$input_path" --output_path "$output_path"


[2025-05-21 04:11:10,813 - INFO]: Scanning images in /content/drive/MyDrive/pbl4/new_set_pkl/078/nm-01/090...
[2025-05-21 04:11:10,822 - INFO]: Found 272 images.
Processing images: 100% 272/272 [00:01<00:00, 163.75it/s]
[2025-05-21 04:11:12,504 - INFO]: Saved 272 images to /content/drive/MyDrive/pbl4/new_pkl/078/nm-01/090/090.pkl
[2025-05-21 04:11:12,834 - INFO]: Scanning images in /content/drive/MyDrive/pbl4/new_set_pkl/078/nm-02/090...
[2025-05-21 04:11:12,844 - INFO]: Found 300 images.
Processing images: 100% 300/300 [00:01<00:00, 172.17it/s]
[2025-05-21 04:11:14,605 - INFO]: Saved 300 images to /content/drive/MyDrive/pbl4/new_pkl/078/nm-02/090/090.pkl
[2025-05-21 04:11:14,948 - INFO]: Scanning images in /content/drive/MyDrive/pbl4/new_set_pkl/078/bg-01/090...
[2025-05-21 04:11:14,950 - INFO]: Found 20 images.
Processing images: 100% 20/20 [00:00<00:00, 245.23it/s]
[2025-05-21 04:11:15,048 - INFO]: Saved 20 images to /content/drive/MyDrive/pbl4/new_pkl/078/bg-01/090/090.pkl
[2025-05

In [ ]:
!gdown --id 1-CSpK6XMcH3EQG6s5IZVm4fGdCxU3rA1 -O /content/data.zip

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-CSpK6XMcH3EQG6s5IZVm4fGdCxU3rA1
From (redirected): https://drive.google.com/uc?id=1-CSpK6XMcH3EQG6s5IZVm4fGdCxU3rA1&confirm=t&uuid=ea014c80-f34d-42d4-ab19-7b3685e2ff79
To: /content/data.zip
100% 356M/356M [00:07<00:00, 48.3MB/s]


In [ ]:
!unzip /content/data.zip -d /content/data


Streaming output truncated to the last 5000 lines.
   creating: /content/data/022/nm-05/108/
   creating: /content/data/022/nm-05/126/
   creating: /content/data/022/nm-05/144/
   creating: /content/data/022/nm-05/162/
   creating: /content/data/022/nm-05/180/
  inflating: /content/data/022/nm-05/036/036.pkl  
  inflating: /content/data/022/nm-05/054/054.pkl  
  inflating: /content/data/022/nm-05/126/126.pkl  
  inflating: /content/data/022/nm-05/180/180.pkl  
  inflating: /content/data/022/nm-05/072/072.pkl  
  inflating: /content/data/022/nm-05/000/000.pkl  
  inflating: /content/data/022/nm-05/018/018.pkl  
  inflating: /content/data/022/nm-05/090/090.pkl  
  inflating: /content/data/022/nm-05/162/162.pkl  
  inflating: /content/data/022/nm-05/144/144.pkl  
  inflating: /content/data/022/nm-05/108/108.pkl  
   creating: /content/data/022/nm-04/000/
   creating: /content/data/022/nm-04/018/
   creating: /content/data/022/nm-04/036/
   creating: /content/data/022/nm-04/054/
   creatin

In [ ]:
# CASIA-B
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-06-06 07:11:31.852229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749193891.875240    8487 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749193891.885300    8487 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-06 07:11:31.912713: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
xFormers not available
xFormers not available
[2025-06-06 07:11:36] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': T

In [ ]:
#Data nhóm Đạt model mới
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-06-08 14:41:28.206225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749393688.225631   36051 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749393688.231524   36051 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-08 14:41:28.252007: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
xFormers not available
xFormers not available
[2025-06-08 14:41:32] [INFO]: {'enable_float16': True, 'restore_ckpt_st

In [ ]:
#Model mới 3 góc nhìn
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-06-10 16:57:18.489783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749574638.522580    7829 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749574638.532664    7829 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 16:57:18.564815: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
xFormers not available
xFormers not available
[2025-06-10 16:57:23] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': T

In [ ]:
# 8 ID
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-06-10 16:45:56.080643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749573956.391008    4784 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749573956.474437    4784 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 16:45:57.118302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
xFormers not available
xFormers not available
[2025-06-10 16:46:30] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': T

In [ ]:
# 5 ID
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-06-10 16:49:13.725514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749574153.759169    5708 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749574153.769087    5708 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 16:49:13.801838: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
xFormers not available
xFormers not available
[2025-06-10 16:49:18] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': T

In [ ]:
# 3 ID
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-06-10 16:49:45.383271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749574185.403033    5870 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749574185.409002    5870 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 16:49:45.428834: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
xFormers not available
xFormers not available
[2025-06-10 16:49:49] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': T

In [ ]:
# Train từ 92000 với dataset mới 75 76 77 78 79
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase train

2025-06-04 02:06:41.572045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749002801.605525   10596 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749002801.616471   10596 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 02:06:41.648485: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
xFormers not available
xFormers not available
[2025-06-04 02:06:46] [INFO]: {'find_unused_parameters': False, 'enable

In [ ]:
# 8 id
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:12:31.172673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747800751.397365   24939 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747800751.458767   24939 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:12:31.930782: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:13:04] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 7 id, bỏ 082
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:14:33.938067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747800873.957924   25526 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747800873.963899   25526 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:14:33.983923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:14:39] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 6 id  bỏ 082 081
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:16:49.345708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801009.366642   26154 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801009.372784   26154 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:16:49.393404: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:16:54] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 5 id bỏ 82 81 80
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:17:16.932751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801036.953064   26294 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801036.959155   26294 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:17:16.980202: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:17:22] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 4 id bỏ 81 82 80 79
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:18:10.629526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801090.649166   26545 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801090.657067   26545 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:18:10.676917: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:18:15] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 3id 75 76 77
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:18:41.446395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801121.465829   26705 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801121.472610   26705 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:18:41.494083: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:18:47] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 2 id 75 76
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:19:07.359564: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801147.379000   26839 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801147.385005   26839 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:19:07.404506: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:19:12] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 1 id 75
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:19:39.261948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801179.281604   26996 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801179.287579   26996 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:19:39.307189: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:19:43] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 1 id 76
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:21:04.573328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801264.593460   27385 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801264.599456   27385 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:21:04.619403: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:21:09] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 1 id 77
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:21:28.538992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801288.572972   27509 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801288.582735   27509 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:21:28.613440: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:21:33] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 1 id 78
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:21:54.612295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801314.631611   27647 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801314.637518   27647 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:21:54.659628: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:21:59] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 1 id 79
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:22:21.145779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801341.165299   27789 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801341.171455   27789 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:22:21.191111: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:22:25] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 1 id 80
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:22:55.460055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801375.495402   27957 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801375.505426   27957 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:22:55.536196: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:23:00] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 1 id 81
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:23:20.315616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801400.348319   28092 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801400.358305   28092 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:23:20.391636: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:23:25] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
# 1 id 82
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-21 04:23:39.303733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747801419.323672   28198 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747801419.329722   28198 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 04:23:39.349792: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-21 04:23:44] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name

In [ ]:
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 opengait/main.py --cfgs ./configs/deepgaitv2/DeepGaitV2_casiab.yaml --phase test

2025-05-16 02:34:30.713419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747362870.733840   21699 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747362870.739948   21699 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-16 02:34:30.761079: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-05-16 02:34:35] [INFO]: {'enable_float16': True, 'restore_ckpt_strict': True, 'restore_hint': 60000, 'save_name